In [2]:
import numpy as np
import random
import time
import utils
import matplotlib.pyplot as plt
from networkMod import Network
import matplotlib.patches as mpatches
from utils import println
import pandas as pd
from sklearn.metrics import confusion_matrix

In [3]:
# - Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# - Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
# - Taka care of the dummy variable trap:
X = X[:, 1:]

# - Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# - Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
# - Initiate the classifier:
input_size = len(X_test[0])
hidden_count = 5
hidden_size = 20
output_size = 2
stupid_reward = 0 # use weighted reward by default
print()
print("Creating a network...")
sampleNetwork = Network(input_size,
                        hidden_size,
                        output_size,
                        hidden_count)
time.sleep(0.5)
print("Done!")
print(sampleNetwork)


# - Prepare the grapth data:
accuracy = []
step = []
Fps  = []
Fns  = []
learning_rate = []
learning_rate.append(accuracy)
learning_rate.append(step)
learning_rate.append(Fps)
learning_rate.append(Fns)


# - Traing the model:
println(5)
epochs = 100
for epoch_numb in range(epochs):
    y_pred = []
    score_list = [0]*100
    for test_numb in range(len(X_train)):
        # - Get input:
        X_train[test_numb]
        input_ = X_train[test_numb]
        
        # - Input to the neural net:
        sampleNetwork.getInputAndPropagate(input_)
        
        # - Get the output:
        output = sampleNetwork.returnOutputPlace()
        y_pred.append(output)
        
        # - Compare with the y_train output:
        if stupid_reward == 1:
            if y_train[test_numb] == output:
                feedback =  0.5
            else:
                feedback = -0.5
        elif stupid_reward == 0:
            if y_train[test_numb] == 1:
                # Max punishment for false positives (max=0.5)
                if output == 1:
                    feedback = 0.5
                else:
                    feedback = -0.5
            if y_train[test_numb] == 0:
                # Adjust smart reward here:
                if output == 0:
                    feedback = 0.17
                else:
                    feedback = -0.17
        
        # - Feedback(Reward):
        sampleNetwork.rewardAndUpdate(feedback)

        # - Calculate score (% of right answers):
        if feedback > 0:
            new_score = 1
        else:
            new_score = 0
        score_list.append(new_score)
        score_list.pop(0)

        # - Every Nth turn:
        Nth_turn = 1000
        if test_numb % Nth_turn == 0 and test_numb != 0 and test_numb != Nth_turn:
        
            # Conf matrix:
            cm = confusion_matrix(y_train[:len(y_pred)], y_pred)
            total_error = (cm[1][0]+cm[0][1]) / np.sum(cm)
            false_positives = cm[1][0]/ (cm[1][0]+cm[1][1])
            false_negatives = cm[0][1]/ (cm[0][0]+cm[0][1])
        
            correctness = np.mean(score_list)
        
            # - Append the learning rate list:
            learning_rate[0].append(total_error)
            try:
                learning_rate[1].append(np.mean(learning_rate[1][-1]+Nth_turn))
            except:
                learning_rate[1].append(0)
            learning_rate[2].append(false_positives)
            learning_rate[3].append(false_negatives)
                
            # - Print data:
            print()
            print("Epoch  :", epoch_numb)
            print("Test   :", test_numb)
            print("Error %:", total_error*100)
            print("- False positives %:", false_positives*100)
            print("- False negatives %:", false_negatives*100)
            
            plt.xlabel('Training sample')
            plt.ylabel('Correctness')
            plt.plot(learning_rate[1], learning_rate[0], color="green")
            plt.plot(learning_rate[1], learning_rate[2], color="red")
            plt.plot(learning_rate[1], learning_rate[3], color="blue")
            green_line = mpatches.Patch(color='green', label='Total Error')
            red_line = mpatches.Patch(color='red', label='False Positives')
            blue_line = mpatches.Patch(color='blue', label='False Negatives')
            plt.legend(handles=[green_line, red_line, blue_line])
            plt.show()


Creating a network...
Done!
R: 11 -> 5x20 -> 2
 
 
 
 
 


In [ ]:
plt.xlabel('Training sample')
plt.ylabel('Correctness')
plt.plot(learning_rate[1], learning_rate[0], color="green")
plt.plot(learning_rate[1], learning_rate[2], color="red")
plt.plot(learning_rate[1], learning_rate[3], color="blue")
green_line = mpatches.Patch(color='green', label='Total Error')
red_line = mpatches.Patch(color='red', label='False Positives')
blue_line = mpatches.Patch(color='blue', label='False Negatives')
plt.legend(handles=[green_line, red_line, blue_line])
plt.show()

Evaluating on the test set:

In [ ]:
y_pred = []
score_list = [0]*100
for test_numb in range(len(X_test)):
    # - Get input:
    X_test[test_numb]
    input_ = X_test[test_numb]
    
    # - Input to the neural net:
    sampleNetwork.getInputAndPropagate(input_)
    
    # - Get the output:
    output = sampleNetwork.returnOutputPlace()
    y_pred.append(output)
    
    # - Compare with the y_train output:
    if y_test[test_numb] == output:
        feedback = 0.5
    else:
        feedback = -0.5
    
    # - Continue learning through the test set?:
    #sampleNetwork.rewardAndUpdate(feedback)
    
    # - Calculate score (% of right answers):
    if feedback == 0.5:
        new_score = 1
    else:
        new_score = 0
    score_list.append(new_score)
    score_list.pop(0)
    
    # - print every 100 turns:
    if test_numb % 100 == 0:
        correctness = np.mean(score_list)
        print(test_numb)

print("Tested network:")
print(sampleNetwork)
print("- Trained for", epochs, "epochs")
print()
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
print(cm)
print()
print("Correct answers in %:", ((cm[0][0]+cm[1][1])/2000.)*100)
print("-  False negatives %:", (cm[0][1] / (cm[0][1]+cm[0][0]))*100)
print("-  False positives %:", (cm[1][0] / (cm[1][1]+cm[1][0]))*100)
print()